# stata_session

> A class for representing a Stata session

Some parts adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py), limited for now to variables, globals, locals, scalars, matrices, and file names.

In [ ]:
#| default_exp stata_session
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.helpers import run_noecho, diverted_stata_output
from fastcore.basics import patch_to
from textwrap import dedent
import re

In [ ]:
#| export
class StataSession():
    def __init__(self):

        self.matchall = re.compile(
            r"\A.*?"
            r"%varlist%(?P<varlist>.*?)"
            r"%globals%(?P<globals>.*?)"
            r"%scalars%(?P<scalars>.*?)"
            r"%matrices%(?P<matrices>.*?)(\Z|---+\s*end)",
            flags=re.DOTALL + re.MULTILINE).match

        # Varlist-style matching; applies to most
        self.varlist = re.compile(r"(?:\s+)(\S+)", flags=re.MULTILINE)

        # file-style matching
        self.filelist = re.compile(r"[\r\n]{1,2}", flags=re.MULTILINE)

        # Clean line-breaks.
        self.varclean = re.compile(
            r"(?=\s*)[\r\n]{1,2}?^>\s", flags=re.MULTILINE).sub

        #         # Match output from mata mata desc
#         self.matadesc = re.compile(
#             r"(\A.*?---+|---+[\r\n]*\Z)", flags=re.MULTILINE + re.DOTALL)

#         self.matalist = re.compile(
#             r"(?:.*?)\s(\S+)\s*$", flags=re.MULTILINE + re.DOTALL)

#         self.mataclean = re.compile(r"\W.*?(\b|$)")
#         self.matasearch = re.compile(r"(?P<kw>\w.*?(?=\W|\b|$))").search

        self.refresh()
#         self.suggestions = self.get_suggestions(kernel)
#         self.suggestions['magics'] = kernel.magics.available_magics
#         self.suggestions['magics_set'] = config.all_settings

    def refresh(self):
        self.suggestions = self.get_suggestions()
#         self.suggestions['magics'] = kernel.magics.available_magics
#         self.suggestions['magics_set'] = config.all_settings
#         self.globals = self.get_globals(kernel)

To take the place of `self.quickdo('_StataKernelCompletions', kernel)`:

In [ ]:
#| export
def variable_names():
    from sfi import Data
    return [Data.getVarName(i) for i in range(Data.getVarCount())]

In [ ]:
#| eval: false
from nbstata.config import launch_stata

In [ ]:
#| eval: false
launch_stata(splash=False)
run_noecho("gen var1 = 1")
variable_names()

['var1']

In [ ]:
#| export
@patch_to(StataSession)
def _completions(self):
#     return dedent(f"""\
#     %varlist%
#     {' '.join(variable_names())}
#     %globals%
#     {' '.join(global_names())}
#     """
    return diverted_stata_output(dedent("""\
        set more off
        set trace off
        syntax [varlist]
        disp "%varlist%"
        disp `"`varlist'"'
        disp "%globals%"
        disp `"`:all globals'"'
        * NOTE: This only works for globals; locals are, well, local ):
        * disp "%locals%"
        * mata : invtokens(st_dir("local", "macro", "*")')
        disp "%scalars%"
        disp `"`:all scalars'"'
        disp "%matrices%"
        disp `"`:all matrices'"'
    """))

In [ ]:
show_doc(StataSession._completions)

---

[source](https://github.com/hugetim/nbstata/blob/main/nbstata/stata_session.py#L62){target="_blank" style="float:right; font-size:smaller"}

### StataSession._completions

>      StataSession._completions ()

In [ ]:
#| export
@patch_to(StataSession)
def get_suggestions(self):
    match = self.matchall(self._completions())
    if match:
        suggestions = match.groupdict()
#         suggestions['mata'] = self._parse_mata_desc(suggestions['mata'])
#         suggestions['programs'] = self._parse_programs_desc(
#             suggestions['programs'])
        for k, v in suggestions.items():
#             if k in ['mata', 'programs']:
#                 continue
#             elif k in ['logfiles']:
#                 suggestions[k] = [
#                     f for f in self.filelist.split(v.strip()) if f]
#             else:
            suggestions[k] = self.varlist.findall(self.varclean('', v))

        all_locals = """mata : invtokens(st_dir("local", "macro", "*")')"""
        res = '\r\n'.join(
            re.split(r'[\r\n]{1,2}', diverted_stata_output(all_locals)))
        if res.strip():
            suggestions['locals'] = self.varlist.findall(
                self.varclean('', res))
        else:
            suggestions['locals'] = []
    else:
        suggestions = {
            'varlist': [],
            'scalars': [],
            'matrices': [],
#             'logfiles': [],
            'globals': [],
#             'programs': [],
            'locals': [],
        }

    return suggestions

In [ ]:
#| eval: false
# kernel1 = Mock()
test_instance = StataSession() #kernel1)
# print(test_instance._completions())
test_instance.suggestions

{'varlist': ['var1'],
 'globals': ['S_level',
  'F1',
  'F2',
  'F7',
  'F8',
  'S_ADO',
  'S_StataMP',
  'S_StataSE',
  'S_CONSOLE',
  'S_FLAVOR',
  'S_OS',
  'S_OSDTL',
  'S_MACH'],
 'scalars': [],
 'matrices': [],
 'locals': []}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()